## Avaliação por meio de um método de aprendizado de máquina

Os embeddings podem oferecer uma informação de proximidade de conceitos que o uso de Bag of Words não seria capaz. Mesmo assim, cada representação e preprocessamento tem sua vantagem e desvantagem e não existe um método que será sempre o melhor. Assim, para sabermos qual representação é melhor para uma tarefa, é importante avaliarmos em quais delas são maiores para a tarefa em questão. Como o foco desta prática não é a avaliação, iremos apenas apresentar o resultado, caso queira, você pode [assistir a video aula](https://www.youtube.com/watch?v=Ag06UuWTsr4&list=PLwIaU1DGYV6tUx10fCTw5aPnqypbbK_GJ&index=12) e [fazer a prática sobre avaliação](https://github.com/daniel-hasan/ap-de-maquina-cefetmg-avaliacao/archive/master.zip). Nesta parte, iremos apenas usar a avaliação para verificar qual método é melhor.  

Para que esta seção seja auto contida, iremos fazer toda a preparação que fizemos nas seções anteriores

**Criação da lista de stopwords e de vocabulário:**

In [1]:
# reset para liberar memória
%reset -f
# fim do reset para liberar memória

from embeddings.utils import get_embedding, KDTreeEmbedding
from sklearn.decomposition import PCA

emotion_words = {
                    "Sexism":{"mulher","homem","cis","trans","feminista","burra","feia","gorda","puta","menina"},
                    "Body":{"gorda", "gordo", "magro", "magra","magrela","magrelo","feio","feia","burra","feminista","mulher"},
                    "Racism":{"negro","negra", "preto", "preta", "racismo", "branco","branca","latino","inverso"},
                    "Ideology":{"esquerda","direita","esquerdopata","esquerdomacho","direitista","minion"},
                    "Homophobia":{"fufa","sapatão","gay","viado","hetero","bicha"},
                    "Origin":{"latino", "brasileiro", "europeu", "gringo"},
                    "Religion":{"muçulmano", "deus", "allah", "crente","cristão","evangélico"},
                    "Health":{"deficiente", "amputado", "amputa","cadeirante"},
                    "OtherLifestyle":{"vegano","vegetariano","hippie","rock","emo","nerd"},
                    "Migrants":{"gringo", "estrangeiro", "Brasil", "Portugal", "Angola"},
                    "Ageing":{"velho","velha","incapaz","senil","idoso","idosa"},
            }
dict_embedding = get_embedding("glove.pt.100.txt")

kdtree_embedding = KDTreeEmbedding(dict_embedding, "kdt_pt.p")

#obtem as stopwords
stop_words = set()
with open("datasets/stopwords.txt") as stop_file:
    stop_words = set(stop_word[:-1] for stop_word in stop_file)


#palavras chaves a serem consideradas
set_vocabulary = set()
for key_word, arr_related_words in emotion_words.items():
    set_vocabulary.add(key_word)
    set_vocabulary = set_vocabulary | set(arr_related_words)

#kdtree - para gerar o conjunto com palavras chaves e suas similares
vocabulary_expanded = []
for word in set_vocabulary:
    _, words = kdtree_embedding.get_most_similar_embedding(word,60)
    vocabulary_expanded.extend(words)
vocabulary_expanded = set(vocabulary_expanded)

LInha com erro: 'afeta -0.536855 -0.007495 -0.013442 0.010075 -0.431695 -0.954242 -0.568022 0.298830 0.206329 0.221990 0.448505 0.324589 0.155598 -0.434498 -0.038841 0.351460 -0.219903 0.292218 -0.116971 0.102685 0.944467 0.388329 -0.330937 -0.755884 -0.164395 0.377288 -0.361163 -0.915998 0.161222 0.827306 -0.284279 0.053623 -0.500227 0.372490 -0.171850 -0.247056 0.115936 -0.017340 -0.118077 -0.008613 0.009058 -0.344892 0.526107 0.021267 0.123609 0.112071 0.277755 -0.655675 0.056385 -0.489364 -0.011241 -0.068256 -0.050418 0.283620 1.146130 -1.045703 0.120836 0.311448 -0.007991 -0.395445 -0.616343 -0.102998 0.801631 0.035789 0.522152 -0.000360 0.081070 0.359324 0.164685 0.103358 -0.434422 -0.047618 0.685093 -0.245462 0.899385 0.430083 -0.097732 -0.991104 0.267290 0.055047 0.469607 -0.454359 -0.206270 -0.075901 -0.702083 -0.149101 0.101842 -0.126275 0.175566 -0.050471 -0.131559 0.382135 -0.021810 -0.609549 0.137217 -0.443079 -0.909590 -0.520087 0.576502 -0.078572
'
Palavras ignoradas: 2




**Representações usadas**:Iremos avaliar a filtragem de stopwords e usando um vocabulário restrito da representação bag of words e também da representação usando a média de embeddings.

In [2]:
from embeddings.textual_representation import BagOfWords, AggregateEmbeddings,InstanceWisePreprocess

#gera as representações
aggregate = AggregateEmbeddings(dict_embedding, "avg")
embedding = InstanceWisePreprocess("embbeding",aggregate)

aggregate_stop = AggregateEmbeddings(dict_embedding, "avg",words_to_filter=stop_words)
emb_nostop = InstanceWisePreprocess("emb_nostop",aggregate_stop)


aggregate_keywords_exp = AggregateEmbeddings(dict_embedding, "avg",words_to_consider=vocabulary_expanded)
emb_keywords_exp = InstanceWisePreprocess("emb_keywords_exp",aggregate_keywords_exp)

bow_keywords = BagOfWords("bow_keywords_exp", words_to_consider=vocabulary_expanded)
bow = BagOfWords("bow", stop_words=stop_words)

arr_representations = [bow, embedding, emb_keywords_exp,bow_keywords]

In [3]:
import pandas as pd
df_hate_speech = pd.read_csv("2021-07-20_portuguese_hate_speech_hierarchical_classification.csv",delimiter=";")

Abaixo, é executado um método de aprendizado  para cada representação. Esse processo pode demorar um pouco pois é feito a procura do melhor parametro do algoritmo. Algumas otimizações que talvez, você precise fazer é no arquivo `embedding/avaliacao_embedding.py` alterar o parametro `n_jobs` no método `obtem_metodo` da classe `OtimizacaoObjetivoRandomForest`. Esse parametro é responsável por utiizar mais threads ao executar o Random Forests.  O valor pode ser levemente inferior a quantidades de núcleos que seu computador tem, caso ele tenha mais de 2, caso contrário, o ideal é colocarmos `n_jobs=1`. Caso queira visualizar resultados mais rapidamente, diminua o valor da variável `num_trials` e `num_folds` abaixo. Atenção que `num_folds` deve ser um valor maior que um.

In [4]:
import pandas as pd
import optuna
from embeddings.avaliacao_embedding import calcula_experimento_representacao, OtimizacaoObjetivoRandomForest

# Método de aprendizado de máquina a ser usado
dict_metodo = {"random_forest":{"classe_otimizacao":OtimizacaoObjetivoRandomForest,
                                "sampler":optuna.samplers.TPESampler(seed=1, n_startup_trials=10)},
              }
df_amazon_reviews = pd.read_csv("2021-07-20_portuguese_hate_speech_hierarchical_classification.csv",delimiter=";")

#executa experimento com a representacao determinada e o método
for metodo, param_metodo in dict_metodo.items():
    for representation in arr_representations:
        print(f"===== Representação: {representation.nome}")
        col_classe = "Hate.speech"
        num_folds = 3
        num_folds_validacao = 3
        num_trials = 50


        nom_experimento = f"{metodo}_"+representation.nome
        experimento = calcula_experimento_representacao(nom_experimento,representation,df_amazon_reviews,
                                            col_classe,num_folds,num_folds_validacao,num_trials,
                                            ClasseObjetivoOtimizacao=param_metodo['classe_otimizacao'],
                                                sampler=param_metodo['sampler'])
        print(f"Representação: {representation.nome} concluida")

===== Representação: bow


[I 2022-01-12 08:37:02,399] A new study created in RDB with name: random_forest_bow_fold_0
[I 2022-01-12 08:42:32,024] Trial 0 finished with value: 0.7923223281565432 and parameters: {'min_samples_split': 9, 'max_features': 95, 'num_arvores': 30}. Best is trial 0 with value: 0.7923223281565432.


KeyboardInterrupt: 

Como a experimentação é uma tarefa custosa, todos os resultados são salvos na pasta "resultados" - inclusive os valores dos parametros na classe optuna (a prática de avaliação apresenta mais detalhes da biblioteca Optuna). A macro f1 é uma métrica relacionada a taxa de acerto (se necessário, [veja a explicação neste video - tópico 2 e 3)](https://www.youtube.com/watch?v=u7o7CSeXaNs&list=PLwIaU1DGYV6tUx10fCTw5aPnqypbbK_GJ&index=13). Analise os resultados abaixo: qual representação foi melhor? A restrição de vocabulário ou eliminação de stopwords auxiliou? 

In [ ]:
import os
import pandas as pd
from base_am.avaliacao import Experimento

arr_resultado = []
for resultado_csv in os.listdir("resultados"):
    if resultado_csv.endswith("csv"):
        nom_experimento = resultado_csv.split(".")[0]
        
        #carrega resultados previamente realizados
        experimento = Experimento(nom_experimento,[])
        experimento.carrega_resultados_existentes()
        
        #adiciona experimento
        num_folds = len(experimento.resultados)
        dict_resultados = {"nom_experimento":nom_experimento, 
                            "macro-f1":sum([r.macro_f1 for r in experimento.resultados])/num_folds}
        #resultados por classe
        for classe in experimento.resultados[0].mat_confusao.keys():

            dict_resultados[f"f1-{classe}"] = sum([r.f1_por_classe[classe] for r in experimento.resultados])/num_folds
            dict_resultados[f"precision-{classe}"] = sum([r.precisao[classe] for r in experimento.resultados])/num_folds
            dict_resultados[f"recall-{classe}"] = sum([r.revocacao[classe] for r in experimento.resultados])/num_folds

        arr_resultado.append(dict_resultados)

resultado = pd.DataFrame.from_dict(arr_resultado)
resultado.sort_values(['macro-f1'], ascending = False, inplace = True)
resultado

### Discussão

Estudando o resultado acima, ordenado em forma decrescente pelo critério macro $F1$ que leva em consideração as predições corretas, os falsos negativos e os falsos positivos do classificador para calcular o quão bom ele é em classificar os individuos de determinados grupos (quanto mais próximo de 1 esse valor for, melhor o classificador)

Assim, verificamos que a representação bag of words é a que apresenta o maior macro $F1$, ou seja, a que mais analisa corretamente os sentimentos das revisões dos usuários. Porém, como dito anteriormente, essa representação é sujeita a uma limitação de generalização, visto que ela não permite calcular a distância entre palavras e consequentemente, a construir estruturas de sinônimos ou analogias por exemplo.

Por fim, random_forest_bow é seguida pela random forest embedding e random forest embedding que desconsidera stopwords. Considerei isso um aspecto interessante, pois, stopwords em tese são palavras sem grande contribuição semântica para a extração do sentimento de sentenças, mas no caso presente, elas dão um incremento significativo (aproximadamente 2%) no macro $F1$ do classificador

## Bibliografia

Bolukbasi, T., Chang, K. W., Zou, J., Saligrama, V., & Kalai, A. (2016). **[Man is to computer programmer as woman is to homemaker? Debiasing word embeddings](https://arxiv.org/abs/1607.06520)**. 

Hartmann, N., Fonseca, E., Shulby, C., Treviso, M., Rodrigues, J., & Aluisio, S. (2017). [**Portuguese word embeddings: Evaluating on word analogies and natural language tasks.**](https://arxiv.org/abs/1708.06025)


Pennington, J., Socher, R., & Manning, C. D. (2014, October).**[GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/pubs/glove.pdf)**. In EMNLP 2015 


Scherer, Klaus R. **[What are emotions? And how can they be measured?](https://journals.sagepub.com/doi/pdf/10.1177/0539018405058216)**. Social science information, v. 44, n. 4, p. 695-729, 2005.

Shen, D., Wang, G., Wang, W., Min, M. R., Su, Q., Zhang, Y., Carin, L. (2018). [Baseline needs more love: On simple word-embedding-based models and associated pooling mechanisms](https://arxiv.org/pdf/1805.09843.pdf).




<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Licença Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />Este obra está licenciado com uma Licença <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Atribuição-CompartilhaIgual 4.0 Internacional</a>.